Copyright **`(c)`** 2025 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [891]:
import numpy as np
import time
import copy
import math
from typing import Callable

In [892]:
NUM_KNAPSACKS = 3
NUM_ITEMS = 10
NUM_DIMENSIONS = 2

In [893]:
VALUES = np.random.randint(0, 100, size=NUM_ITEMS)
WEIGHTS = np.random.randint(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = np.random.randint(
    0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

In [894]:
CONSTRAINTS

array([[ 60, 155],
       [ 69,  61],
       [ 81,  65]], dtype=int32)

## TEST PROBLEMS

In [895]:
# Problem 1:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

In [896]:
from dataclasses import dataclass


@dataclass
class Solution:
    knapsacks: list[set[int]]
    total_value: int

    def get_items_left(self) -> set[int]:
        items_left = set(range(NUM_ITEMS))
        for ks in self.knapsacks:
            items_left -= ks
        return items_left

In [897]:
def find_first_solution(
    knapsacks: list[set[int]] = None,
) -> Solution:
    if knapsacks is None:
        knapsacks = [set() for _ in range(NUM_KNAPSACKS)]

    running_totals = np.zeros((NUM_KNAPSACKS, NUM_DIMENSIONS), dtype=int)
    total_value = 0

    # We build a simple probability distribution that favors items with higher value-to-weight ratios.
    nudged_probs = np.array(
        [
            1e-3 + np.log(VALUES[i] + 1) - (np.log((WEIGHTS[i] + 1)).sum())
            for i in range(NUM_ITEMS)
        ]
    )

    free_items = set(range(NUM_ITEMS))
    for ks in knapsacks:
        free_items -= ks

    while len(free_items) > 0:
        probs = nudged_probs[list(free_items)]
        probs = probs / probs.sum()
        item = rng.choice(list(free_items), p=probs)

        # Calculate availability after adding the item to each knapsack
        availability_after = CONSTRAINTS - (running_totals + WEIGHTS[item])
        # print((availability_after / CONSTRAINTS).shape)
        # Find knapsacks that can actually accommodate the item
        available_knapsacks = np.where((availability_after >= 0).all(axis=1))[0]
        # print(available_knapsacks)
        # If no knapsack can accommodate the item, skip it
        if len(available_knapsacks) == 0:
            # This item cannot be added to any knapsack (we may be in a local optimum)
            free_items.remove(item)
            continue

        if len(available_knapsacks) == 1:
            knapsack_i = available_knapsacks[0]

        else:
            balance_after = (
                CONSTRAINTS[available_knapsacks]
                - availability_after[available_knapsacks]
            )
            balance_after = balance_after.max(axis=1) - balance_after.min(axis=1)
            nudged_probs2 = 1 / (balance_after + 4)
            nudged_probs2 = nudged_probs2 / nudged_probs2.sum()
            # print(balance_after, balance_after.shape)
            # Select the knapsack that will have the most balanced load between the dimensions of constraint after adding the item
            knapsack_i = available_knapsacks[balance_after.argmin()]
            # knapsack_i = rng.choice(available_knapsacks, p=nudged_probs2)

        knapsacks[knapsack_i].add(item)
        free_items.remove(item)
        running_totals[knapsack_i] += WEIGHTS[item]
        total_value += VALUES[item]

    return Solution(knapsacks=knapsacks, total_value=total_value)

In [898]:
first_solution = find_first_solution()
print(first_solution.total_value)

1020


In [899]:
def benchmark_solution(
    solution_factory: Callable[[], Solution], n_runs: int = 10
) -> Solution:
    start_time = time.time()

    runs_points = np.zeros(n_runs)
    best_knapsacks: list[set[int]] = []
    best_value = 0

    for run_i in range(n_runs):
        solution = solution_factory()
        total_value = solution.total_value

        runs_points[run_i] = total_value
        if total_value > best_value:
            best_value = total_value
            best_knapsacks = solution.knapsacks

    end_time = time.time()
    avg_time = (end_time - start_time) / n_runs

    print("Best total value found:", max(runs_points))
    # print("Best knapsacks found:", best_knapsacks)

    print(f"Statistics over runs (n = {n_runs}):")
    print(f"  Min: {runs_points.min():.2f}")
    print(f"  Max: {runs_points.max():.2f}")
    print(f"  Mean: {runs_points.mean():.2f}")
    print(f"  Std: {runs_points.std():.2f}")
    print(f"Elapsed time: {(avg_time):.4f} seconds")

    return Solution(knapsacks=best_knapsacks, total_value=best_value)

In [900]:
benchmarked_solution = benchmark_solution(find_first_solution, n_runs=10)

Best total value found: 1065.0
Statistics over runs (n = 10):
  Min: 926.00
  Max: 1065.00
  Mean: 1001.70
  Std: 45.98
Elapsed time: 0.0018 seconds


In [901]:
# Shuffle random items between knapsacks
def tweak_solution(original: Solution) -> Solution:
    new_solution = copy.deepcopy(original)

    running_totals = np.zeros((NUM_KNAPSACKS, NUM_DIMENSIONS), dtype=int)
    for knapsack_i in range(NUM_KNAPSACKS):
        running_totals[knapsack_i] += WEIGHTS[
            list(new_solution.knapsacks[knapsack_i])
        ].sum(axis=0)

    # Decide how many items to tweak
    n_tweaks = rng.integers(1, max(2, NUM_ITEMS // 20))

    for _ in range(n_tweaks):
        # Randomly choose an item to tweak
        item = rng.integers(0, NUM_ITEMS)

        # Find which knapsack currently contains the item (if any)
        current_knapsack_i = None
        for knapsack_i in range(NUM_KNAPSACKS):
            if item in new_solution.knapsacks[knapsack_i]:
                current_knapsack_i = knapsack_i
                break

        # Remove the item from its current knapsack (if any)
        if current_knapsack_i is not None:
            new_solution.knapsacks[current_knapsack_i].remove(item)
            running_totals[current_knapsack_i] -= WEIGHTS[item]
            new_solution.total_value -= VALUES[item]

        # Try to add the item to a different knapsack
        available_knapsacks = []
        for knapsack_i in range(NUM_KNAPSACKS):
            if knapsack_i != current_knapsack_i:
                availability_after = CONSTRAINTS[knapsack_i] - (
                    running_totals[knapsack_i] + WEIGHTS[item]
                )
                if (availability_after >= 0).all():
                    available_knapsacks.append(knapsack_i)

        if len(available_knapsacks) > 0:
            chosen_knapsack_i = rng.choice(available_knapsacks)
            new_solution.knapsacks[chosen_knapsack_i].add(item)
            running_totals[chosen_knapsack_i] += WEIGHTS[item]
            new_solution.total_value += VALUES[item]

    return new_solution

In [902]:
tweak_solution(benchmarked_solution).total_value

np.int64(1013)

In [903]:
def benchmark_tweak(
    original: Solution,
    tweak_function: Callable[[Solution], Solution],
    n_runs: int = 10,
) -> Solution:
    start_time = time.time()

    runs_points = np.zeros(n_runs)
    best_solution = original

    for run_i in range(n_runs):
        solution = tweak_function(original)

        runs_points[run_i] = solution.total_value
        if solution.total_value > best_solution.total_value:
            best_solution = solution

    end_time = time.time()
    avg_time = (end_time - start_time) / n_runs

    print("Best total value found after tweak:", max(runs_points))
    # print("Best knapsacks found:", best_knapsacks)

    print(f"Statistics over tweak runs (n = {n_runs}):")
    print(f"  Min: {runs_points.min():.2f}")
    print(f"  Max: {runs_points.max():.2f}")
    print(f"  Mean: {runs_points.mean():.2f}")
    print(f"  Std: {runs_points.std():.2f}")
    print(f"Elapsed time: {(avg_time):.4f} seconds")

    return best_solution

In [904]:
tweaked_solution = benchmark_tweak(benchmarked_solution, tweak_solution, n_runs=10)

Best total value found after tweak: 1053.0
Statistics over tweak runs (n = 10):
  Min: 980.00
  Max: 1053.00
  Mean: 1013.90
  Std: 18.64
Elapsed time: 0.0002 seconds


In [905]:
tweaked_solution = benchmark_tweak(tweaked_solution, tweak_solution, n_runs=100)

Best total value found after tweak: 1065.0
Statistics over tweak runs (n = 100):
  Min: 968.00
  Max: 1065.00
  Mean: 1022.20
  Std: 31.10
Elapsed time: 0.0002 seconds


In [906]:
from tqdm import tqdm

In [910]:
for _ in tqdm(range(10)):
    tweaked_solution = benchmark_tweak(tweaked_solution, tweak_solution, n_runs=100)

100%|██████████| 10/10 [00:00<00:00, 73.70it/s]

Best total value found after tweak: 1065.0
Statistics over tweak runs (n = 100):
  Min: 968.00
  Max: 1065.00
  Mean: 1021.58
  Std: 32.84
Elapsed time: 0.0001 seconds
Best total value found after tweak: 1065.0
Statistics over tweak runs (n = 100):
  Min: 968.00
  Max: 1065.00
  Mean: 1023.13
  Std: 32.60
Elapsed time: 0.0001 seconds
Best total value found after tweak: 1065.0
Statistics over tweak runs (n = 100):
  Min: 968.00
  Max: 1065.00
  Mean: 1025.49
  Std: 32.69
Elapsed time: 0.0001 seconds
Best total value found after tweak: 1065.0
Statistics over tweak runs (n = 100):
  Min: 968.00
  Max: 1065.00
  Mean: 1021.84
  Std: 31.69
Elapsed time: 0.0002 seconds
Best total value found after tweak: 1065.0
Statistics over tweak runs (n = 100):
  Min: 968.00
  Max: 1065.00
  Mean: 1022.30
  Std: 31.69
Elapsed time: 0.0001 seconds
Best total value found after tweak: 1065.0
Statistics over tweak runs (n = 100):
  Min: 968.00
  Max: 1065.00
  Mean: 1024.58
  Std: 32.78
Elapsed time: 0.0001 

## TEST PROBLEMS

In [908]:
# Problem 1:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

In [ ]:
first_solution = benchmark_solution(find_first_solution, n_runs=100)
first_solution.total_value
tweaked_solution = first_solution
for _ in tqdm(range(10)):
    tweaked_solution = benchmark_tweak(tweaked_solution, tweak_solution, n_runs=100)
print(tweaked_solution.total_value)

Best total value found: 1065.0
Statistics over runs (n = 100):
  Min: 807.00
  Max: 1065.00
  Mean: 983.10
  Std: 59.37
Elapsed time: 0.0013 seconds


100%|██████████| 10/10 [00:00<00:00, 81.17it/s]

Best total value found after tweak: 1065.0
Statistics over tweak runs (n = 100):
  Min: 968.00
  Max: 1065.00
  Mean: 1008.91
  Std: 25.91
Elapsed time: 0.0001 seconds
Best total value found after tweak: 1065.0
Statistics over tweak runs (n = 100):
  Min: 968.00
  Max: 1065.00
  Mean: 1011.48
  Std: 28.22
Elapsed time: 0.0001 seconds
Best total value found after tweak: 1065.0
Statistics over tweak runs (n = 100):
  Min: 968.00
  Max: 1065.00
  Mean: 1015.29
  Std: 29.51
Elapsed time: 0.0001 seconds
Best total value found after tweak: 1065.0
Statistics over tweak runs (n = 100):
  Min: 968.00
  Max: 1065.00
  Mean: 1013.64
  Std: 30.83
Elapsed time: 0.0001 seconds
Best total value found after tweak: 1065.0
Statistics over tweak runs (n = 100):
  Min: 968.00
  Max: 1065.00
  Mean: 1011.02
  Std: 26.21
Elapsed time: 0.0001 seconds
Best total value found after tweak: 1065.0
Statistics over tweak runs (n = 100):
  Min: 968.00
  Max: 1065.00
  Mean: 1010.77
  Std: 27.45
Elapsed time: 0.0001 

In [912]:
# Problem 2:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    1000 * 2, 1000 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

In [918]:
first_solution = benchmark_solution(find_first_solution, n_runs=1000)
first_solution.total_value
tweaked_solution = first_solution
for _ in tqdm(range(250)):
    tweaked_solution = benchmark_tweak(tweaked_solution, tweak_solution, n_runs=250)
print(tweaked_solution.total_value)

Best total value found: 38784.0
Statistics over runs (n = 1000):
  Min: 28950.00
  Max: 38784.00
  Mean: 34313.93
  Std: 1696.01
Elapsed time: 0.0065 seconds


  0%|          | 1/250 [00:00<00:26,  9.39it/s]

Best total value found after tweak: 39614.0
Statistics over tweak runs (n = 250):
  Min: 36024.00
  Max: 39614.00
  Mean: 38056.77
  Std: 586.69
Elapsed time: 0.0004 seconds


  1%|          | 2/250 [00:00<00:26,  9.50it/s]

Best total value found after tweak: 40052.0
Statistics over tweak runs (n = 250):
  Min: 36959.00
  Max: 40052.00
  Mean: 38812.77
  Std: 631.82
Elapsed time: 0.0004 seconds
Best total value found after tweak: 40501.0
Statistics over tweak runs (n = 250):
  Min: 36824.00
  Max: 40501.00
  Mean: 39248.77
  Std: 612.27
Elapsed time: 0.0004 seconds


...

Best total value found after tweak: 49782.0
Statistics over tweak runs (n = 250):
  Min: 46514.00
  Max: 49782.00
  Mean: 48644.43
  Std: 634.28
Elapsed time: 0.0004 seconds
Best total value found after tweak: 49782.0
Statistics over tweak runs (n = 250):
  Min: 46958.00
  Max: 49782.00
  Mean: 48685.93
  Std: 668.78
Elapsed time: 0.0004 seconds


 99%|█████████▉| 248/250 [00:29<00:00,  9.07it/s]

Best total value found after tweak: 49782.0
Statistics over tweak runs (n = 250):
  Min: 46385.00
  Max: 49782.00
  Mean: 48679.02
  Std: 646.35
Elapsed time: 0.0004 seconds
Best total value found after tweak: 49782.0
Statistics over tweak runs (n = 250):
  Min: 46524.00
  Max: 49782.00
  Mean: 48668.73
  Std: 697.87
Elapsed time: 0.0004 seconds


100%|██████████| 250/250 [00:30<00:00,  8.32it/s]

Best total value found after tweak: 49782.0
Statistics over tweak runs (n = 250):
  Min: 46594.00
  Max: 49782.00
  Mean: 48755.10
  Std: 660.31
Elapsed time: 0.0004 seconds
Best total value found after tweak: 49782.0
Statistics over tweak runs (n = 250):
  Min: 47047.00
  Max: 49782.00
  Mean: 48754.05
  Std: 639.51
Elapsed time: 0.0004 seconds
49782


In [919]:
# Problem 3:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 100
NUM_ITEMS = 5000
NUM_DIMENSIONS = 100
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    1000 * 10,
    1000 * 2 * NUM_ITEMS // NUM_KNAPSACKS,
    size=(NUM_KNAPSACKS, NUM_DIMENSIONS),
)

In [921]:
first_solution = benchmark_solution(find_first_solution, n_runs=10)
first_solution.total_value
tweaked_solution = first_solution
for _ in tqdm(range(50)):
    tweaked_solution = benchmark_tweak(tweaked_solution, tweak_solution, n_runs=50)
print(tweaked_solution.total_value)

Best total value found: 1227842.0
Statistics over runs (n = 10):
  Min: 1178363.00
  Max: 1227842.00
  Mean: 1210011.80
  Std: 14642.08
Elapsed time: 2.5133 seconds


  2%|▏         | 1/50 [00:04<03:24,  4.18s/it]

Best total value found after tweak: 1246227.0
Statistics over tweak runs (n = 50):
  Min: 1220891.00
  Max: 1246227.00
  Mean: 1231176.46
  Std: 6221.96
Elapsed time: 0.0835 seconds


  4%|▍         | 2/50 [00:08<03:23,  4.24s/it]

Best total value found after tweak: 1269573.0
Statistics over tweak runs (n = 50):
  Min: 1246633.00
  Max: 1269573.00
  Mean: 1257671.24
  Std: 6290.67
Elapsed time: 0.0856 seconds


...

Best total value found after tweak: 1691319.0
Statistics over tweak runs (n = 50):
  Min: 1678558.00
  Max: 1691319.00
  Mean: 1685676.68
  Std: 2945.14
Elapsed time: 0.0845 seconds


 98%|█████████▊| 49/50 [03:21<00:04,  4.25s/it]

Best total value found after tweak: 1696169.0
Statistics over tweak runs (n = 50):
  Min: 1678152.00
  Max: 1696169.00
  Mean: 1687745.66
  Std: 3642.85
Elapsed time: 0.0922 seconds


100%|██████████| 50/50 [03:25<00:00,  4.12s/it]

Best total value found after tweak: 1700867.0
Statistics over tweak runs (n = 50):
  Min: 1682583.00
  Max: 1700867.00
  Mean: 1694242.54
  Std: 3410.91
Elapsed time: 0.0786 seconds
1700867
